In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("PopularHero")
sc = SparkContext(conf = conf)

In [3]:
def countCoOccurences(line):
    elements = line.split()
    return (int(elements[0]), len(elements) - 1)

In [4]:
def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode('utf8'))

In [5]:
names = sc.textFile("../data/Marvel-Names.txt")

In [7]:
names.collect()[:5]

['1 "24-HOUR MAN/EMMANUEL"',
 '2 "3-D MAN/CHARLES CHAN"',
 '3 "4-D MAN/MERCURIO"',
 '4 "8-BALL/"',
 '5 "A"']

In [8]:
namesRdd = names.map(parseNames)

In [9]:
lines = sc.textFile("../data/Marvel-Graph.txt")

pairings = lines.map(countCoOccurences)

In [11]:
pairings.collect()[:5]

[(5988, 48), (5989, 40), (5982, 42), (5983, 14), (5980, 24)]

In [12]:
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)

In [13]:
totalFriendsByCharacter.collect()[:5]

[(1, 5), (2, 122), (3, 72), (4, 14), (5, 8)]

In [15]:
flipped = totalFriendsByCharacter.map(lambda xy: (xy[1], xy[0]))

In [16]:
flipped.collect()[:5]

[(5, 1), (122, 2), (72, 3), (14, 4), (8, 5)]

In [17]:
mostPopular = flipped.max()

In [19]:
mostPopular

(1933, 859)

In [20]:
mostPopularName = namesRdd.lookup(mostPopular[1])[0]

In [21]:
print(str(mostPopularName) + " is the most popular superhero, with " + \
    str(mostPopular[0]) + " co-appearances.")

b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.
